In [ ]:
import torch
import numpy as np
import pandas as pd
import os

from data_factory import *
from model_factory import *

In [ ]:
## This code is for training the main Transformer model

# Load training and validation datasets as Pytorch Dataset objects
train_dataset = DelaneyDataset(mode = "train")
valid_dataset = DelaneyDataset(mode = "valid")


In [ ]:
# Create instance of Transformer model
model = ChemTransformer().cuda()

# ModelFactory() is a helper class for managing the model
model_factory = ModelFactory(model)

criterion = nn.MSELoss()
metric = nn.MSELoss()

# Load saved weights
#model_factory.load()

In [ ]:
# Define optimizer and scheduler for training
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5, patience = 25, verbose = True)

In [ ]:
# epochs = number of epochs to train

epochs = 400

for e in range(epochs):
  # Train once through the entire training set
  train_loss = train(model_factory, train_dataset, criterion, optimizer)
  # Valid once
  valid(model_factory, valid_dataset, metric)

  # Print train and validation results from current epoch
  model_factory.print_last_loss(epoch = e)
  # Save only the best result based on cross-validation
  model_factory.save_best("loss_valid")

  scheduler.step(train_loss)